In [173]:
import pandas as pd
import pickle 
from scipy.stats import poisson
from numpy import sqrt
import math

In [3]:
dict_table = pickle.load(open('dict_table.pickle','rb'))
historical_data = pd.read_csv('Clean_fifa_worldcup_matches.csv')
fixture = pd.read_csv('Clean_fifa_worldcup_fixture.csv')

In [298]:
home = historical_data[['HomeTeam','HomeGoals','AwayGoals']]
away = historical_data[['AwayTeam','HomeGoals','AwayGoals']]
home = home.rename(columns={'HomeTeam':'Team','HomeGoals':'ScoredGoals','AwayGoals':'ConcededGoals'})
away = away.rename(columns={'AwayTeam':'Team','HomeGoals':'ConcededGoals','AwayGoals':'ScoredGoals'})
#team_strength = pd.concat([home,away],ignore_index=True).groupby('Team').mean()

In [299]:
Actuality = ((1.1-(2022 - historical_data['Year'])/92)**4)*3
Actuality2 = pd.concat([Actuality,Actuality],ignore_index=True)
WeightedGoals = pd.concat([home,away],ignore_index=True)
WeightedGoals['ScoredGoals'] = WeightedGoals['ScoredGoals'] * Actuality2
WeightedGoals['ConcededGoals'] = WeightedGoals['ConcededGoals'] * Actuality2
team_strength = WeightedGoals.groupby('Team').mean()
team_strength

,ScoredGoals,ConcededGoals
Team,,
Algeria,1.985486,2.626201
Angola,0.735541,1.471083
Argentina,1.948283,1.292430
Australia,2.246327,4.871874
Austria,0.465253,0.580829
...,...,...
Uruguay,1.466196,1.371197
Wales,0.064155,0.064155
West Germany,0.739700,0.405408


In [307]:
def predict_points(Home,Away):
    if Home  in team_strength.index and Away in team_strength.index:
        lamb_home = (team_strength.at[Home,'ScoredGoals']*team_strength.at[Away,'ConcededGoals'])
        lamb_away = (team_strength.at[Away,'ScoredGoals']*team_strength.at[Home,'ConcededGoals'])
        prob_home, prob_away   , prob_draw = 0,0,0
        for x in range(0,10):
            for y in range(0,10):
                p = poisson.pmf(x,lamb_home)*poisson.pmf(y,lamb_away)
                if x == y:
                    prob_draw += p
                elif x>y:
                    prob_home += p
                elif y>x:
                    prob_away += p
        points_home = 3*prob_home + prob_draw
        point_away = 3*prob_away + prob_draw
        #return(points_home,points_away)
        return(prob_home*100,prob_draw*100,prob_away*100)
    else:
        return(0,0)

In [308]:
a,b,c = predict_points('Argentina','Netherlands')
print('Arg:',a.round(1),"%   ",'Empate:',b.round(1),'%   ','Neth',c.round(1),'%')

Arg: 26.0 %    Empate: 15.8 %    Neth 58.0 %


In [309]:
a,b,c = predict_points('France','England')
print('Fra:',a.round(1),"%   ",'Empate:',b.round(1),'%   ','Eng',c.round(1),'%')

Fra: 57.4 %    Empate: 17.6 %    Eng 24.9 %


In [310]:
a,b,c = predict_points('Brazil','Croatia')
print('Bra:',a.round(1),"%   ",'Empate:',b.round(1),'%   ','Cro',c.round(1),'%')

Bra: 58.3 %    Empate: 10.5 %    Cro 20.9 %


In [311]:
a,b,c = predict_points('Morocco','Portugal')
print('Mor:',a.round(1),"%   ",'Empate:',b.round(1),'%   ','Por',c.round(1),'%')

Mor: 13.8 %    Empate: 9.5 %    Por 71.1 %
